In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/tent

/content/drive/MyDrive/tent


In [ ]:
!pip install torchattacks
!!pip install -r requirements.txt
#One important thing, remove the first three lines from requirements.txt, no need to downgrade torch and torchvision

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchattacks
from torchattacks import PGD, FGSM

In [ ]:
#Feel free to play around with this model or add custom model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5), # 16*24*24
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16,32,5), # 32*20*20
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2), # 32*10*10
            nn.Conv2d(32,64,5), # 64*6*6
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2) #64*3*3
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(-1,64*3*3)
        out = self.fc_layer(out)

        return out


In [ ]:
#Dataset
mnist_train = dsets.MNIST(root='./data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='./data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
batch_size = 128

train_loader  = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                         batch_size=batch_size,
                                         shuffle=False)

In [ ]:
model = CNN().cuda()

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs=5
atk = PGD(model, eps=0.3, alpha=0.1, steps=7)

In [ ]:
for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = atk(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/5], lter [100/468], Loss: 1.5526
Epoch [1/5], lter [200/468], Loss: 0.8477
Epoch [1/5], lter [300/468], Loss: 0.6861
Epoch [1/5], lter [400/468], Loss: 0.4772
Epoch [2/5], lter [100/468], Loss: 0.3729
Epoch [2/5], lter [200/468], Loss: 0.3412
Epoch [2/5], lter [300/468], Loss: 0.3874
Epoch [2/5], lter [400/468], Loss: 0.2129
Epoch [3/5], lter [100/468], Loss: 0.3142
Epoch [3/5], lter [200/468], Loss: 0.2840
Epoch [3/5], lter [300/468], Loss: 0.2953
Epoch [3/5], lter [400/468], Loss: 0.1122
Epoch [4/5], lter [100/468], Loss: 0.2499
Epoch [4/5], lter [200/468], Loss: 0.2466
Epoch [4/5], lter [300/468], Loss: 0.2440
Epoch [4/5], lter [400/468], Loss: 0.0839
Epoch [5/5], lter [100/468], Loss: 0.2125
Epoch [5/5], lter [200/468], Loss: 0.2382
Epoch [5/5], lter [300/468], Loss: 0.2526
Epoch [5/5], lter [400/468], Loss: 0.0766


In [ ]:
#Evaluate accuracy
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

Standard accuracy: 98.81 %


In [ ]:
#Evaluate Robust Accuracy
model.eval()

correct = 0
total = 0

atk = FGSM(model, eps=0.3)

for images, labels in test_loader:
    
    images = atk(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 92.20 %


In [ ]:
import tent
model = tent.configure_model(model)
params, param_names = tent.collect_params(model)
tented_model = tent.Tent(model, optimizer)

In [ ]:
#Tenting forward passes and updates using test data-set
for images, labels in test_loader:
    images = images.cuda()
    outputs = tented_model(images)

In [ ]:
#Accuracy after tenting
tented_model.eval()
correct = 0
total = 0
for images, labels in test_loader:
    images=images.cuda()
    outputs = tented_model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
print('Accuracy after Tenting: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 98.79 %
